## Основные этапы работы:

### Этап 0. Установка и настройка оболочки для работы с языком Python

В лабораторной работе использовалсь среда разработки VS Code с расширением Juputer <u>version: 2024.2.0</u>.

Загрузка необходимых библиотек для выполнения лаборатрных работ:

In [57]:
import tensorflow as tf
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras.utils import to_categorical

В лабораторной работе используется в обучении нейросети Keras и TensorFlow.
Keras — это библиотека уровня модели, предоставляющая высокоуровневые строительные блоки для конструирования моделей глубокого обучения.

### Этап 1. Построение бинарного классификатора.
**Целью этапа:** является создание бинарного классификатора отзывов к фильмам из наборы данных IMDB. 

**Формулировка задания:** классифицировать отзывы к фильмам на положительные и отрицательные отзывы, опираясь на текст отзывов.

Вариант этапа 1

| № | Количество слоев | Количество нейронов на слое | Функции активации скрытого слоя | Функция потерь |
| :-: | :-: | :-: | :-: | :-: |
| 3 | 2 | 32 | tanh-relu | Бинарная перекрестная энтропия |

Пошаговая реализация поставленной цели включает:

	1. Загрузка набора данных IMDB

In [2]:
from tensorflow.keras.datasets import imdb

База данных состоит из 50000 отзывов к кинолентам в интернет-базе (Internet Movie Database). Набор разбит на 25000 обучающих и 25 000 контрольных отзывов, каждый набор на 50 % состоит из отрицательных и на 50 % из положительных отзывов. Набор данных IMDB поставляется в составе Keras. Набор готов к использованию: отзывы (последовательности слов) преобразованы в последовательности целых чисел, каждое из которых определяет позицию слова в словаре.

	2. Разделение данных на обучающий и тестовый наборы

In [7]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

Аргумент _num_words=10000_ означает, что в обучающих данных будет сохранено только 10000 слов, наиболее часто встречающихся в обучающем наборе отзывов, остальные слова будут отброшены.

In [10]:
train_data

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

Переменные _train_data_ и _test_data_ — это списки отзывов; каждый отзыв — это список индексов слов (кодированное представление последовательности слов).

In [11]:
train_labels

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

Переменные _train_labels_ и _test_labels_ — это списки нулей и единиц, где нули соответствуют отрицательным отзывам, а единицы — положительным.

	3. Подготовка данных для передачи в нейронную сеть

Выполним прямое кодирование списков в векторы нулей и единиц. Это преобразование последовательности, например, [3, 5] в 10000-мерный вектор, все элементы которого содержат нули, кроме элементов с индексами 3 и 5, которые содержат единицы. Затем их можно передать в первый слой сети типа Dense, способный обрабатывать векторизованные данные с вещественными числами.

In [18]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))#1
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.#2
    return results 

Векторизуем обучающие и контрольные данные:

In [19]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

Посмотрим на примере первого образца как выглядят теперь данные:

In [20]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

Векторизуем метки:

In [21]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

    4. В соответствии с вариантом, выполнить конструирование сети: выбрать количество скрытых слоев, количество нейронов в каждом слое, вид активационной функции на каждом слое

С помощью функции _Sequential()_ мы создаём новую модель. Сеть будет состоять из 3 полносвязных слоёв Dense. Первый слой состоит из 64 нейронов, с функцией активацией _tanh_, и количеством в 10000 значений на входе сети. Второй слой тоже состоит из 64 нейронов, с функцией активацией _relu_. Последний слой состоит из одного нейрона с функцией активации _sigmoid_, на выходе получаем скалярное значение в диапозоне между 0 и 1, представляющее собой вероятность

In [62]:
model = models.Sequential()
model.add(layers.Dense(64, activation='tanh', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Выведем архитектуру сети:

В первом слое 640064 параметров, во втором 4160, в последнем 65 параметров. Всего в модели 644289 обучающихся параметров.

In [64]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 64)                640064    
                                                                 
 dense_46 (Dense)            (None, 64)                4160      
                                                                 
 dense_47 (Dense)            (None, 1)                 65        
                                                                 
Total params: 644289 (2.46 MB)
Trainable params: 644289 (2.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Входные данные представлены векторами, а метки — скалярами (единицами и нулями). С задачами этого
вида прекрасно справляются сети, организованные как простой стек полносвязных (Dense) слоев. Аргумент, передаваемый каждому слою Dense, — это число скрытых нейронов слоя.
Скрытый нейрон (hidden unit) - это измерение в пространстве представлений слоя. Наличие 32 скрытых нейронов означает, что весовая матрица W будет иметь форму (input_dimension, 32): скалярное произведение на W спроецирует входные данные в 16-мерное пространство представлений (затем будет произведено сложение с вектором смещений b и выполнена операция relu), каждый слой Dense с операцией активации relu реализует следующую цепочку операций с тензорами:

<p style="text-align: center;">output=relu(dot(W, input) + b)</p>

Функция relu (rectified linear unit — блок линейной ректификации) используется для преобразования отрицательных значений в ноль. 

<center><img src="relu.jpg"></center>
<center>Рис. 1. Функция блока линейной ректификации</center>

Сигмоидная функция рассредоточивает произвольные значения по интервалу [0,1], возвращая значения, которые можно интерпретировать как вероятность.
<center><img src="sigmoid.jpg"></center>
<center>Рис. 2. Сигмоидная функция</center>

    5. Настройка оптимизатора с выбором функции потерь и метрики качества

In [49]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Перекрестная энтропия (crossentropy) - это мера расстояния между распределениями вероятностей, или в данном случае - между фактическими данными и предсказаниями.

Настраиваем модель оптимизатором rmsprop и функцией потерь mеап squared error.

rmsprop - наиболее подходящий оптимизатор, популярный в использовании для большинства нейронных сетей.

Среднеквадратичное распространение корня (RMSprop) - это экспоненциально затухающее среднее значение. Существенным свойством RMSprop является то, что вы не ограничены только суммой прошлых градиентов, но вы более ограничены градиентами последних временных шагов. В RMSProp мы пытаемся уменьшить вертикальное движение, используя среднее значение, потому что они суммируются приблизительно до 0, принимая среднее значение. RMSprop предоставляет среднее значение для обновления.

    6. Проведение проверки решения, выделяя контрольное множество

Для контроля точности выделим проверочное множество 10000 образцов из обучающего множества данных:

In [30]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

Обучим модель на 20 эпохах пакетами (batch_size) по 512 образцов:

In [31]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/20
25/30 [========================>.....] - ETA: 0s - loss: 0.5177 - accuracy: 0.7342

30/30 [==============================] - 2s 44ms/step - loss: 0.4919 - accuracy: 0.7550 - val_loss: 0.3343 - val_accuracy: 0.8691
Epoch 2/20
30/30 [==============================] - 0s 13ms/step - loss: 0.2855 - accuracy: 0.8876 - val_loss: 0.3747 - val_accuracy: 0.8391
Epoch 3/20
30/30 [==============================] - 0s 14ms/step - loss: 0.2237 - accuracy: 0.9132 - val_loss: 0.2772 - val_accuracy: 0.8842
Epoch 4/20
30/30 [==============================] - 0s 13ms/step - loss: 0.1672 - accuracy: 0.9369 - val_loss: 0.5290 - val_accuracy: 0.8187
Epoch 5/20
30/30 [==============================] - 0s 13ms/step - loss: 0.1452 - accuracy: 0.9442 - val_loss: 0.3305 - val_accuracy: 0.8790
Epoch 6/20
30/30 [==============================] - 0s 13ms/step - loss: 0.1188 - accuracy: 0.9570 - val_loss: 0.3868 - val_accuracy: 0.8601
Epoch 7/20
30/30 [==============================] - 0s 13ms/step - loss: 0.1004 - accuracy: 0.9637 - val_loss: 0.3516 - val_accuracy: 0.8803
Epoch 8/20
30/30 [======

model.fit() возвращает объект History. Этот объект имеет поле history — словарь с данными обо всем происходившем в процессе обучения.

    7. Вывод графиков функции потерь и точности

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.clf()
plt.plot(epochs, loss_values, 'r', label='Потери на этапе обучения')
plt.plot(epochs, val_loss_values, 'b', label='Потери на этапе проверки')
plt.title('Потери на этапах обучения и проверки')
plt.xlabel('Эпохи')
plt.ylabel('Потери')
plt.legend()
plt.show()

<center><img src="1.1.3.png"></center>
<center>Рис. 3. Потери на этапах обучения и проверки</center>

In [ ]:
plt.clf()
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
plt.plot(epochs, acc, 'r', label='Точность на этапе обучения')
plt.plot(epochs, val_acc, 'b', label='Точность на этапе проверки')
plt.title('Точность на этапах обучения и проверки')
plt.xlabel('Эпохи')
plt.ylabel('Точность')
plt.legend()
plt.show()

<center><img src="1.1.2.png"></center>
<center>Рис. 4. Точность на этапах обучения и проверки</center>

Посмотрим какой точности достигнет наша модель обученная на 20 эпохах:

In [34]:
results = model.evaluate(x_test, y_test)

782/782 [==============================] - 1s 1ms/step - loss: 0.7744 - accuracy: 0.8560


По итогам обучения с 20 эпохами точность достигла 85,47%.

На этапе обучения потери снижаются с каждой эпохой, а точность растет. Это ожидаемое поведение от оптимизации градиентным спуском: величина, которую необходимо минимизировать, становиться все меньше с каждой итерацией на обучающих данных. 
Однако потери и точность на этапе проверки достигает пика в третью эпоху. Далее происходит переобучение: после третьей эпохи произошла чрезмерная оптимизация на обучающих данных, и в результате получилось представление, характерное для обучающих данных, не обобщающее данные за пределами обучающего набора.

В данном случае для предотвращения переобучения можно прекратить обучение после третьей эпохи.

Скомпилируем заново модель, ограничимся тремя эпохами и проверим точность на контрольных данных.

In [41]:
model = keras.Sequential([
  layers.Dense(64, activation='tanh'),
  layers.Dense(64, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=512)

Epoch 1/3
49/49 [==============================] - 1s 8ms/step - loss: 0.4351 - accuracy: 0.7991
Epoch 2/3
49/49 [==============================] - 0s 8ms/step - loss: 0.2544 - accuracy: 0.8985
Epoch 3/3
49/49 [==============================] - 0s 8ms/step - loss: 0.2032 - accuracy: 0.9207


In [42]:
results_end = model.evaluate(x_test, y_test)
results_end

782/782 [==============================] - 2s 2ms/step - loss: 0.2912 - accuracy: 0.8806


[0.29117682576179504, 0.8806399703025818]

Модель обученная в течении трёх эпох показала точность в 88,06% при потерях 0.29.

Посмотрим сколько мы выигрываем при обучении модели на двадцати эпохах и модели обученной на трех эпохах:

In [44]:
print(results_end[0] - results[0])
print(results_end[1] - results[1])

-0.4832030236721039
0.024599969387054443


Мы получаем не только выигрыш по времени, но и улучшаем точность на 2,46% и потери стали меньше на 0.48.

    8. Использование обученной сети для предсказания на новых данных

Попробуем предсказать вероятность того, что отзывы будут положительными, с помощью метода predict:


In [49]:
model.predict(x_test)

782/782 [==============================] - 1s 1ms/step


array([[0.12984616],
       [0.9984404 ],
       [0.29253614],
       ...,
       [0.05470785],
       [0.03435762],
       [0.47761816]], dtype=float32)

Рузультат показывает, что сеть уверена в одних образцах (99% или 3%), но в других не так точно (47,8%)

### Этап 2. Построение многоклассого классификатора.
**Целью этапа:** является создание многоклассового классификатора новостных лент из набора данных Reuters. 

**Формулировка задания:** создать сеть для классификации новостных лент агентства Reuters на 46 взаимоисключающих тем. 

Вариант этапа 2

| № | Количество слоев | Количество нейронов на слое | Функции активации скрытого слоя | Функция потерь |
| :-: | :-: | :-: | :-: | :-: |
| 3 | 2 | 4-32 | tanh-relu | многокатегориальная перекрестная энтропия |

Пошаговая реализация поставленной цели включает:

	1. Загрузка набора данных Reuters

In [32]:
from keras.datasets import reuters

По сравнению с предыдущем этапом лабораторной работы классов больше двух, значит эта задача относится к категории задач многоклассовой классификации; и, поскольку каждый экземпляр данных должен быть отнесен только к одному классу, эта задача является примером однозначной многоклассовой классификации. 

Reuters - простой набор данных, широко используемых для классификации текста. Существует 46 разных тем; некоторые темы более широко представлены, некоторые менее, но для каждой из них в обучающем наборе имеется не менее 10 примеров.

	2. Разделение данных на обучающий и тестовый наборы

In [33]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

Аргумент _num_words=10000_ по аналогии с предыдущем этапом означает, что в обучающих данных будет сохранено только 10000 слов, остальные слова будут отброшены.

In [34]:
train_data

array([list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]),
       list([1, 3267, 699, 3434, 2295, 56, 2, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 2, 49, 2295, 2, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 2, 2, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12]),
       list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 3886, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 1013, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90, 67, 960, 4, 78, 13, 68, 467, 511, 110,

Переменные _train_data_ и _test_data_ — это списки новостных лент, каждая новость — это список индексов слов (кодированное представление последовательности слов).

In [35]:
train_labels

array([ 3,  4,  3, ..., 25,  3, 25], dtype=int64)

Переменные _train_labels_ и _test_labels_ — это списки меток определяющий класс принадлежности новостных лент от 0 до 45.

	3. Подготовка данных для передачи в нейронную сеть

Декодируем первую новость в списке тренировочных данных и выведем её метку:

In [5]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

print(decoded_newswire)

? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


В decoded_newswire мы сместили индексы на 3, так как они зарезервировали их на отступ, начало последовательности, неизвестно.

Используем функцию _vectorize_sequences_, которая была написана на предыдущем этапе и выполним прямое кодирование списков в векторы из нулей и единиц:

In [6]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

Для меток используем прямое кодирование (one-hot encoding). Прямое кодирование широко используется для форматирования категорий и также называется кодированием категорий (categorical encoding):

In [7]:
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

    4. В соответствии с вариантом, выполнить конструирование сети: выбрать количество скрытых слоев, количество нейронов в каждом слое, вид активационной функции на каждом слое

С помощью функции _Sequential()_ мы создаём новую модель. Сеть будет состоять из 3 полносвязных слоёв Dense. Первый слой состоит из 4 нейронов, с функцией активацией _tanh_, и количеством в 10000 значений на входе сети. Второй слой тоже состоит из 32 нейронов, с функцией активацией _relu_. Последний слой состоит из 46 нейронов с функцией активации _softmax_, возвращающий массив с 10 оценками вероятностей (в сумме дающий 1), каждая оценка определяет вероятность принадлежности к одному из 46 классов.

In [65]:
model = keras.Sequential([
  layers.Dense(4, activation='tanh', input_shape=(10000,)),
  layers.Dense(32, activation='relu'),
  layers.Dense(46, activation='softmax')
])

В отличие от предыдущей сети, эта завершается слоем Dense с размером 46. Это означает, что для каждого входного образца сеть будет выводить 46-мерный вектор. Каждый элемент этого вектора (каждое измерение) представляет собой отдельный выходной класс.

Последний слой использует функцию активации softmax. Означает, что сеть будет выводить распределение вероятностей по 46 разным классам - для каждого образца на входе сеть будет возвращать 46-мерный вектор, где output[i] вероятность принадлежности образца классу і. Сумма 46 элементов всегда будет равна 1.

Выведем архитектуру сети:

В первом слое 40004 параметров, во втором 160, в последнем 1518 параметров. Всего в модели 41682 обучающихся параметров.

In [66]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 4)                 40004     
                                                                 
 dense_49 (Dense)            (None, 32)                160       
                                                                 
 dense_50 (Dense)            (None, 46)                1518      
                                                                 
Total params: 41682 (162.82 KB)
Trainable params: 41682 (162.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


    5. Настройка оптимизатора с выбором функции потерь и метрики качества
    
В данном случае используем функцию потерь categorical_crossentropy. Она определяет расстояние между распределениями вероятностей. В данном случае между распределением вероятности на выходе сети и истинным распределением меток. Минимизируя расстояние между этими двумя распределениями, мы учим сеть выводить результат, максимально близкий к истинным меткам.
Оптимизатор поставим такой же как и на прошлом этапе.

Скомпелируем модель:

In [9]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    6. Проведение проверки решения, выделяя контрольное множество

Для контроля точности модели создадим проверочный набор (x_val, y_val), выбрав 1000 образцов из набора обучающих данных.

In [10]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

Обучим модель на 30 эпохах.

In [11]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/30




16/16 [==============================] - 1s 20ms/step - loss: 3.6735 - accuracy: 0.2746 - val_loss: 3.4650 - val_accuracy: 0.3630
Epoch 2/30
16/16 [==============================] - 0s 8ms/step - loss: 3.3245 - accuracy: 0.3926 - val_loss: 3.1478 - val_accuracy: 0.4370
Epoch 3/30
16/16 [==============================] - 0s 8ms/step - loss: 3.0183 - accuracy: 0.4417 - val_loss: 2.8548 - val_accuracy: 0.4170
Epoch 4/30
16/16 [==============================] - 0s 8ms/step - loss: 2.7292 - accuracy: 0.4073 - val_loss: 2.5745 - val_accuracy: 0.4110
Epoch 5/30
16/16 [==============================] - 0s 7ms/step - loss: 2.4590 - accuracy: 0.4110 - val_loss: 2.3228 - val_accuracy: 0.4330
Epoch 6/30
16/16 [==============================] - 0s 8ms/step - loss: 2.2240 - accuracy: 0.4553 - val_loss: 2.1133 - val_accuracy: 0.4990
Epoch 7/30
16/16 [==============================] - 0s 7ms/step - loss: 2.0274 - accuracy: 0.5362 - val_loss: 1.9413 - val_accuracy: 0.5650
Epoch 8/30
16/16 [============

Посмотрим какой точности достигнет наша модель обученная на 30 эпохах:

In [12]:
results = model.evaluate(x_test, one_hot_test_labels)
results

71/71 [==============================] - 0s 890us/step - loss: 1.2485 - accuracy: 0.7012


[1.2484803199768066, 0.7012466788291931]

Модель обученная в течении тридцати эпох показала точность в 70,12% при потерях 1.25.

    7. Вывод графиков функции потерь и точности

In [ ]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'r', label='Потери на этапе обучения')
plt.plot(epochs, val_loss, 'b', label='Потери на этапе проверки')
plt.title('Потери на этапах обучения и проверки')
plt.xlabel('Эпохи')
plt.ylabel('Потери')
plt.legend()
plt.show()

<center><img src="1.2.11.png"></center>
<center>Рис. 5. Потери на этапах обучения и проверки</center>

In [ ]:
plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'r', label='Точность на этапе обучения')
plt.plot(epochs, val_acc, 'b', label='Точность на этапе проверки')
plt.title('Точность на этапах обучения и проверки')
plt.xlabel('Эпохи')
plt.ylabel('Точность')
plt.legend()
plt.show()


<center><img src="1.2.12.png"></center>
<center>Рис. 6. Точность на этапах обучения и проверки</center>

Как можно увидеть точность на валидационном наборе данных увеличивается, и ошибка уменьшается на протяжении 30 эпохах, поэтому попробуем увеличить количество эпох.

Скомпелируем модель заново и обучим её на 50 эпохах.

In [23]:
model = keras.Sequential([
  layers.Dense(4, activation='tanh', input_shape=(10000,)),
  layers.Dense(32, activation='relu'),
  layers.Dense(46, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=50,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/50
11/16 [===================>..........] - ETA: 0s - loss: 3.7091 - accuracy: 0.1042

16/16 [==============================] - 1s 17ms/step - loss: 3.6614 - accuracy: 0.1847 - val_loss: 3.4947 - val_accuracy: 0.4370
Epoch 2/50
16/16 [==============================] - 0s 8ms/step - loss: 3.3543 - accuracy: 0.4921 - val_loss: 3.2079 - val_accuracy: 0.5200
Epoch 3/50
16/16 [==============================] - 0s 7ms/step - loss: 3.0650 - accuracy: 0.5180 - val_loss: 2.9297 - val_accuracy: 0.5300
Epoch 4/50
16/16 [==============================] - 0s 7ms/step - loss: 2.7854 - accuracy: 0.5318 - val_loss: 2.6628 - val_accuracy: 0.5410
Epoch 5/50
16/16 [==============================] - 0s 10ms/step - loss: 2.5238 - accuracy: 0.5380 - val_loss: 2.4190 - val_accuracy: 0.5410
Epoch 6/50
16/16 [==============================] - 0s 8ms/step - loss: 2.2930 - accuracy: 0.5361 - val_loss: 2.2130 - val_accuracy: 0.5460
Epoch 7/50
16/16 [==============================] - 0s 8ms/step - loss: 2.1048 - accuracy: 0.5408 - val_loss: 2.0497 - val_accuracy: 0.5480
Epoch 8/50
16/16 [===========

In [25]:
results = model.evaluate(x_test, one_hot_test_labels)
results

71/71 [==============================] - 0s 1ms/step - loss: 1.4151 - accuracy: 0.7191


[1.4150570631027222, 0.7190561294555664]

Модель обученная в течении тридцати эпох показала точность в 71,91% при потерях 1.41.

Можно заметить, что лучшая точность наступает на 43 эпохе, после просходит переобучение. Скомпелируем модель заново и обучим её на 43 эпохах.

In [28]:
model = keras.Sequential([
  layers.Dense(4, activation='tanh', input_shape=(10000,)),
  layers.Dense(32, activation='relu'),
  layers.Dense(46, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=43,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/43
11/16 [===================>..........] - ETA: 0s - loss: 3.6770 - accuracy: 0.0707

16/16 [==============================] - 1s 16ms/step - loss: 3.6143 - accuracy: 0.0941 - val_loss: 3.4009 - val_accuracy: 0.1220
Epoch 2/43
16/16 [==============================] - 0s 7ms/step - loss: 3.2419 - accuracy: 0.3316 - val_loss: 3.0716 - val_accuracy: 0.4570
Epoch 3/43
16/16 [==============================] - 0s 8ms/step - loss: 2.9248 - accuracy: 0.4558 - val_loss: 2.7703 - val_accuracy: 0.4270
Epoch 4/43
16/16 [==============================] - 0s 7ms/step - loss: 2.6374 - accuracy: 0.4213 - val_loss: 2.5034 - val_accuracy: 0.4140
Epoch 5/43
16/16 [==============================] - 0s 7ms/step - loss: 2.3953 - accuracy: 0.4084 - val_loss: 2.2903 - val_accuracy: 0.4030
Epoch 6/43
16/16 [==============================] - 0s 8ms/step - loss: 2.2045 - accuracy: 0.4153 - val_loss: 2.1206 - val_accuracy: 0.4130
Epoch 7/43
16/16 [==============================] - 0s 7ms/step - loss: 2.0465 - accuracy: 0.4459 - val_loss: 1.9790 - val_accuracy: 0.4700
Epoch 8/43
16/16 [============

In [29]:
results = model.evaluate(x_test,one_hot_test_labels)

71/71 [==============================] - 0s 817us/step - loss: 1.2998 - accuracy: 0.7061


Модель показала точность в 71%. Такая низкая точность обусловлена скорее всего малым количеством нейронов модели

    8. Использование обученной сети для предсказания на новых данных

In [30]:
predictions = model.predict(x_test)
# Each entry in predictions is a vector of length 46
print(predictions)

71/71 [==============================] - 0s 848us/step
[[1.2209652e-07 1.9807216e-04 3.1894984e-04 ... 2.5865191e-07
  1.2041477e-06 1.6102700e-07]
 [1.7734680e-01 4.0573901e-03 4.3901903e-04 ... 1.1148362e-03
  8.2914706e-04 1.3830791e-03]
 [2.1863871e-03 4.1649085e-02 2.4244498e-01 ... 7.8543660e-04
  5.3814379e-03 9.6492312e-04]
 ...
 [7.5594414e-07 6.4406160e-04 1.7554855e-03 ... 1.4080886e-06
  8.4369512e-06 8.0663460e-07]
 [4.0884866e-04 2.0724341e-02 4.0543254e-02 ... 4.2099136e-04
  4.8568300e-03 2.7555783e-04]
 [4.5317758e-04 8.7876379e-01 4.9817539e-03 ... 1.1842228e-05
  1.7861741e-04 2.7488170e-05]]


Каждый элемент в predictions - это вектор с длиной 46. Сумма коэффициентов этого вектора равна 1. Предсказание для каждого класса - это вероятность для каждого из 46 элементов и в сумме составляет 1.
Наибольший элемент, элемент с наибольшей вероятностью, — это предсказанный класс.

### Этап 3. Построение прогноза на основе регрессионной модели
**Целью этапа:** является создание нейронной сети, дающей прогноз цен на дома из набора данных Boston Housing.

**Формулировка задания:** построить регрессионную модель для предсказания медианной цены на дома в пригороде Бостона.

Вариант этапа 3

| № | Количество слоев | Количество нейронов на слое | Функции активации скрытого слоя | Функция потерь |
| :-: | :-: | :-: | :-: | :-: |
| 3 | 2 | 64-64 | tanh-tanh | среднеквадратическая погрешность |

Пошаговая реализация поставленной цели включает:

	1. Загрузка набора данных Boston Housing

In [37]:
from keras.datasets import boston_housing

В этом этапе мы попытаемся предсказать медианную цену на дома в пригороде Бостона в середине 1970 года. Для этого используем регрессионную модель.
Основное отличие скалярной регрессии от двух предыдущих этапов, заключается в предсказании не дискретной метки, а значения на непрерывной числовой прямой: например, температуры воздуха на завтра по имеющимся метеорологическим данным или предсказание времени завершения программного проекта по его спецификациям.

Для предсказания воспользуемся данными, как уровень преступности, ставка местного имущественного налога и т. д. Используемый набор данных, имеет интересное отличие от двух предыдущих этапов. Он содержит относительно немного образцов данных: всего 506, разбитых на 404 обучающих и 102 контрольных образца. И каждый признак во входных данных (например, уровень преступности) имеет свой масштаб. Например, некоторые признаки являются пропорциями и имеют значения между 0 и 1, другие - между 1 и 12 и т.д.

	2. Разделение данных на обучающий и тестовый наборы

В наборе данных содержится 404 обучающих и 102 контрольных образца, каждый с 13 числовыми признаками.

In [38]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Вид тестового набора данных представляет собой медианные значения цен на дома, занимаемые собственниками, в тысячах долларов:

In [39]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

	3. Подготовка данных для передачи в нейронную сеть

Можно увидеть, что значения имеют самые разные диапазоны, сеть, конечно может справится, однако это усложнит обучение, обычно в таких случаях принимают нормализацию. Суть нормализации состоит в том что для каждого признака во входных данных (столбца в матрице входных данных) из каждого значения вычитается среднее по этому признаку, и разность делится на стандартное отклонение, в результате признак центрируется по нулевому значению и имеет стандартное отклонение, равное единице.

In [5]:
mean=train_data.mean(axis=0)
train_data-=mean
std=train_data.std(axis=0)
train_data/=std
test_data-=mean
test_data/=std

    4. В соответствии с вариантом, выполнить конструирование сети: выбрать количество скрытых слоев, количество нейронов в каждом слое, вид активационной функции на каждом слое
    
    5. Настройка оптимизатора с выбором функции потерь и метрики качества

Сеть состоит из двух скрытых слоев с функциями активации tanh и по 64 нейрона и заканчивается одномерным слоем (линейным слоем), не имеющим функции активации, т.к. функция активации могла бы ограничить диапозон выходных значений. Это конфигурация для скалярной регрессии, целью которой является предсказание одного значения на непрерывной числовой прямой. В данном случае, с линейным последним слоем, сеть способна предсказывать значения из любого диапазона.

Скомпелировали модель с помощью функции _compile_(). Параметры были выбраны следующие: оптимизатор - rmsprop, функция потерь: mse (mean squared error, в перводе, среднеквадратичная ошибка), вычисляющей квадрат разности между предсказанными и целевыми значениями. Для мониторинга этапов обучения используем mae (mean absolute error) - средняя абсолютная ошибка. Это абсолютное значение разности предсказанными и целевыми значениями.

In [59]:
def build_model():
  model=models.Sequential()
  model.add(layers.Dense(64,activation='tanh',input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64,activation='tanh'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
  return model

Выведем архитектуру сети:

В первом слое 896 параметров, во втором 4160, в последнем 65 параметров. Всего в модели 5121 обучающихся параметров.

In [61]:
model = build_model()
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 64)                896       
                                                                 
 dense_43 (Dense)            (None, 64)                4160      
                                                                 
 dense_44 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5121 (20.00 KB)
Trainable params: 5121 (20.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


    6. Проведение проверки решения, используя метод перекрестной проверки по К блокам. Выбрать количество блоков К как количество слоев плюс один
    
Набор данных содержит малое количество экземпляров, соответственно проверочных данных всего 102, это очень мало. Следовательно мы не можем объективно оценить этапы обучения модели, потому как оценки при проверке могут сильно варивораться в зависимости от того, какие данные попадут в проверочный и обучающий наборы.

Для решения этой проблемы используем перекрестную проверку по K блокам (K-fold cross-validation).
Суть проверки по К блокам заключается в разделении доступных данных на К блоков, создании К идентичных моделей и обучении каждой на К-1 блоках с оценкой по оставшимся блокам. По полученным К оценкам вычисляется среднее значение, которое принимается как оценка модели. На рисунке 7 изображен пример разбиения данных на 3 блока.

<center><img src="1.3.1.png"></center>
<center>Рис. 7. Перекрестная проверка по трем блокам</center>

Реализация перекрестной проверки по K блокам написана в функции check():

In [7]:
def check():
    val_data = train_data[i * num_val_samples : (i + 1) * num_val_samples] # Подготовка проверочных данных из блока с номером k
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate( # Подготовка обучающих данных из остальных блоков
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    return val_data, val_targets, partial_train_data, partial_train_targets


По варианту задания количество блоков зададим _K = 3_.

Обучим модель на 100 эпохах.

In [8]:
k=3
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores =[]
for i in range(k):
    print(f"Processing fold #{i}")
    val_data, val_targets, partial_train_data, partial_train_targets = check()
    model = build_model() # Конструирование модели
    model.fit(partial_train_data, partial_train_targets, # Обучение модели (в режиме без вывода сообщений, verbose = 0)
              epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0) # Оценка модели по проверочным данным
    all_scores.append(val_mae)

Processing fold #0






Processing fold #1
Processing fold #2


Выведем среднюю ошибку трех прогонов:

In [9]:
print(all_scores)
np.mean(all_scores)

[2.3323349952697754, 2.50785493850708, 2.8250732421875]


2.5550877253214517

Средняя оценка ошибки прогнозирования составляет 2555 долларов США.

Увеличим количество эпох до 500:

In [10]:
num_epochs = 500 #
all_mae_histories =[]
for i in range(k):
    val_data, val_targets, partial_train_data, partial_train_targets = check()
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=16, verbose=0)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

Вычислим средние значения метрики mae для всех прогонов:

In [11]:
average_mae_history=[np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

    7. Вывод графиков функции потерь и точности

In [ ]:
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel("Эпохи")
plt.ylabel("Оценка MAE")
plt.show()

<center><img src="1.3.2.png"></center>
<center>Рис. 8.  Оценки MAE по эпохам</center>

Из-за проблем с масштабированием и относительно высокой дисперсии опустим первые 60 замеров из-за проблем с масштабированием и заменим оценки экспоненциально скользящим средним по предыдущим оценкам.

Формирование графика с оценками проверок за исключением первых 60 замеров реализована в функции _smooth_curve()_:

In [14]:
def smooth_curve(points, factor=0.9):
    smoothed_points=[]
    for point in points:
      if smoothed_points:
        previous=smoothed_points[-1]
        smoothed_points.append(previous*factor+point*(1-factor))
      else:
        smoothed_points.append(point)
    return smoothed_points


In [ ]:
truncated_mae_history = smooth_curve(average_mae_history[60:])
plt.plot(range(1, len(truncated_mae_history) + 1), truncated_mae_history)
plt.xlabel("Эпохи")
plt.ylabel("Оценки MAE")
plt.show()

<center><img src="1.3.3.png"></center>
<center>Рис. 9. Оценки MAE по эпохам за исключением первых 60 замеров</center>

Согласно этому графику, наилучшая оценка МАЕ достигается где-то на 150 эпохе. После этого момента начинается переобучение. Обучим модель на 150 эпох, только на всем объеме обучающих данных:

In [26]:
model = build_model() # Получить новую скомпилированную модель
model.fit(train_data, train_targets, 
          epochs=150, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

4/4 [==============================] - 0s 2ms/step - loss: 14.4584 - mae: 2.3720


In [27]:
test_mae_score

2.3719701766967773

Средняя оценка ошибки прогнозирования составляет около 2372 долларов США.

<p style="text-align: center;">Заключение</p>

1. На первом этапе мы классифицировали отзывы к фильмам на положительные и отрицательные опираясь на текст отзывов. Использовали набор данных IMDB с 50000 отзывами. Подготовили данные для передачи в сеть с помощью прямого кодирования списков, т.е. закодировали последовательности целых чисел в бинарную матрицу. Сконструировали сеть согласно варианту, состоящую из двух слоев, с 64 нейронами, функции активации скрытых слоев tanh-relu, на выходном слое Dense была использована функция sigmoid, которая на выходе дает скалярное значение в диапозоне от 0 до 1, была использована бинарная перекрестная энтропия, как функция потерь. Скомпелировали и обучили модель сначала на 20 эпохах, но появляется эффект переобучения сети после третьей эпохи, потом переобучили сеть на трех эпохах, в итоге получили модель показывающаяся точность в 88.06% при потерях в 0.29. Также предсказали вероятность пернадлежности тестовых данных.

2. На втором этапе мы классифицировали новостные ленты по их темам, опираясь на их текст. Использовали набор данных Reuters c 11228 новостей. Подготовили данные для передачи в сеть с помощью прямого кодирования как в предыдущем этапе лабораторной работы. Сконструировали сеть согласно варианту, состоящую из двух слоев, на первом слое было использовано 4 нейрона, на втором слое использовано 32 нейрона, функции активации скрытых слоев tanh-relu, на выходном слое была использована функция softmax для распределения вероятностей определения класса новосных лент, на этом этапе была использована многокатегориальная перекрестная энтропия, как функция потерь, которая определяет расстояние между распределениями вероятностей. Для нахождения оптимального количество эпох обучения было выбрано сначало 30 эпох, однако результаты и графики показали, что этого было недостаточно, потом обучили на 50 эпохах, в итоге после 43 эпохи начилось переобучение модели. Обучив модель на 43 эпохах получили точность в 71%, при потерях 1.41. Такая малая точность обусловлена малым количеством нейронов на слоях модели. Также предсказали вероятность пернадлежности тестовых данных к каждому из классов новостных лент.

3. В третьем этапе была решена задача регрессии, она выполняется с применением иных функций потерь, нежели классификация. В этой задаче Использовали набор данных boston_housing, с количеством 506 экземпляров. Обработали данные с помощью нормализации. Сконструировали сеть согласно варианту, состоящую из двух скрытых слоев, с функциями активации tanh и по 64 нейрона и заканчивается одномерным слоем (линейным слоем), не имеющим функции активации. Получается, сеть с линейным последним слоем, сеть способна предсказывать значения из любого диапазона. Скомпелировали модель с парматрами: оптимизатор - rmsprop, функция потерь: mse (среднеквадратичная ошибка) и метрику оценки mae (средняя абсолютная ошибка). Из-за малого количества данных boston_housing решили надежно оценить качество модели с помощью метода перекрестной проверки по K блокам. Обучили модель на 500 эпохах, однако при оценке обучения из-за проблем с масштабированием, а также ввиду относительно высокой дисперсии затруднительно увидеть общую тенденцию. Для оптимизации были опущены первые 60 замеров, которые имеют другой масштаб, отличный от масштаба остальной кривой, а также каждая оценка была заменена экспоненциальным скользящим средним по предыдущим оценкам. В результате переобучили модель на 150 эпох, в итоге средняя оценка ошибки прогнозирования составляет около 2372 долларов США.

<p style="text-align: center;">Список использованной литературы</p>
    1. Шолле Франсуа. Глубокое обучение на Python. - СПб.: Питер, 2018. - 400 с.: ил. - (Серия «Библиотека программиста»).